In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent  # Adjust if your notebook is in src/rag/
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))


In [32]:
from entities.persona import PersonaManager

In [40]:
PersonaManager().get_persona_by_id("trust_builder").description

'Skeptical, analytical, seeks logic and proof before belief'